In [4]:
# !pip install torch==2.3.0 torchtext==0.18.0

In [5]:
# !pip install torchdata

In [6]:
# import torch

# from torchtext.datasets import IMDB

In [7]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [8]:
import datasets
import torch
from torch.utils.data import Dataset, DataLoader

In [9]:
from datasets import load_dataset

# Load the IMDB dataset
imdb_dataset = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
train_dataset = imdb_dataset["train"]
test_dataset = imdb_dataset["test"]

In [11]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [12]:
train_dataset[0]['text']

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [13]:
import os
import re
import string
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove punctuation (optional, based on your needs)
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text


In [34]:
train_preprocessed=train_dataset.map(lambda x: {'text': preprocess_text(x['text'])})
valid_preprocessed=test_dataset.map(lambda x: {'text': preprocess_text(x['text'])})


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [15]:
train_preprocessed[0]['text']

'i rented i am curiousyellow from my video store because of all the controversy that surrounded it when it was first released in 1967 i also heard that at first it was seized by us customs if it ever tried to enter this country therefore being a fan of films considered controversial i really had to see this for myselfthe plot is centered around a young swedish drama student named lena who wants to learn everything she can about life in particular she wants to focus her attentions to making some sort of documentary on what the average swede thought about certain political issues such as the vietnam war and race issues in the united states in between asking politicians and ordinary denizens of stockholm about their opinions on politics she has sex with her drama teacher classmates and married menwhat kills me about i am curiousyellow is that 40 years ago this was considered pornographic really the sex and nudity scenes are few and far between even then its not shot like some cheaply made

In [16]:
class CustomTokenizer:
    def __init__(self, vocab, max_length=512):
        """
        Initialize the tokenizer with a vocabulary and an optional max length for sequences.
        Args:
            vocab (dict): A dictionary mapping tokens to indices.
            max_length (int, optional): Maximum length of the tokenized sequences. Defaults to 512.
        """
        self.vocab = vocab
        self.unk_token = "<UNK>"
        self.max_length = max_length

    def tokenize(self, text):
        """
        Tokenizes the text into words.
        Args:
            text (str): Input text string.
        Returns:
            List[str]: List of tokens.
        """
        return text.lower().split()

    def encode(self, text):
        """
        Converts text to a list of token indices, with truncation and padding.
        Args:
            text (str): Input text string.
        Returns:
            List[int]: List of token indices.
        """
        tokens = self.tokenize(text)
        encoded = [self.vocab.get(token, self.vocab[self.unk_token]) for token in tokens]

        # Truncating if the sequence is too long
        if len(encoded) > self.max_length:
            encoded = encoded[:self.max_length]

        # Padding if the sequence is too short
        padding_length = self.max_length - len(encoded)
        if padding_length > 0:
            encoded.extend([self.vocab.get(self.unk_token)] * padding_length)

        return encoded

    def decode(self, indices):
        """
        Converts a list of indices back to text.
        Args:
            indices (List[int]): List of token indices.
        Returns:
            str: Decoded text string.
        """
        reverse_vocab = {idx: token for token, idx in self.vocab.items()}
        return " ".join([reverse_vocab.get(idx, self.unk_token) for idx in indices])


In [17]:
from collections import Counter



def build_vocab(texts, min_freq=3, special_tokens=None):
    """
    Build a vocabulary from a list of texts.
    Args:
        texts (List[str]): List of text samples.
        min_freq (int): Minimum frequency for a token to be included in the vocabulary.
        special_tokens (List[str]): List of special tokens (e.g., <PAD>, <UNK>, <CLS>).
    Returns:
        dict: Vocabulary mapping tokens to indices.
    """
    # Count token frequencies
    token_counter = Counter()
    for text in texts:
        tokens = text.lower().split()  # Simple whitespace tokenization
        token_counter.update(tokens)

    # Include only tokens with frequency >= min_freq
    vocab_tokens = [token for token, freq in token_counter.items() if freq >= min_freq]

    # Add special tokens if provided
    if special_tokens is None:
        special_tokens = ["<PAD>", "<UNK>"]
    vocab_tokens = special_tokens + vocab_tokens

    # Create the vocabulary
    vocab = {token: idx for idx, token in enumerate(vocab_tokens)}
    return vocab


In [18]:
vocab=build_vocab(train_preprocessed['text'],special_tokens=["<PAD>", "<UNK>"])

In [19]:
len(vocab)

42784

In [20]:
tokenizer = CustomTokenizer(vocab)

In [21]:
# # Example text
# text = "Hello world, this is a custom token"

# # Tokenize text
# tokens = tokenizer.tokenize(text)
# print("Tokens:", tokens)

# # Encode text
# indices = tokenizer.encode(text)
# print("Indices:", indices)

# # Decode indices
# decoded_text = tokenizer.decode(indices)
# print("Decoded Text:", decoded_text)


In [22]:
class CustomImdbDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        text = self.dataset[index]["text"]
        label = self.dataset[index]["label"]

        # Tokenizing the text and applying padding/truncation
        encoded_text = self.tokenizer.encode(text)

        return {
            "input_ids": torch.tensor(encoded_text, dtype=torch.long),
            "label": torch.tensor(label, dtype=torch.long),
        }


In [35]:
train_Data=CustomImdbDataset(train_preprocessed,tokenizer)
test_Data=CustomImdbDataset(valid_preprocessed,tokenizer)


In [24]:
def custom_collate_fn(batch):
    """
    Custom collate function to pad sequences in a batch.
    Args:
        batch (List[dict]): List of dictionary items with 'input_ids' and 'label'.
    Returns:
        dict: Batch with padded sequences and labels.
    """
    # Extract input_ids and labels
    input_ids = [item["input_ids"] for item in batch]
    labels = [item["label"] for item in batch]

    # Find the maximum sequence length in the batch
    max_len = max(len(seq) for seq in input_ids)

    # Pad sequences to the maximum length
    padded_input_ids = torch.stack([
        torch.cat([seq, torch.full((max_len - len(seq),), 0, dtype=torch.long)])
        for seq in input_ids
    ])

    # Stack labels
    labels = torch.stack(labels)

    return {"input_ids": padded_input_ids, "label": labels}


In [25]:
train_dataloader=DataLoader(train_Data,batch_size=32,shuffle=True,collate_fn=custom_collate_fn)
test_dataloader=DataLoader(test_Data,batch_size=32,shuffle=True,collate_fn=custom_collate_fn)

In [26]:
# Create dataset and dataloader

# Iterate through the dataloader
for batch in train_dataloader:
    print("Batchtrai input_ids:\n", batch["input_ids"])
    print("Batch labels:", batch["label"])
    break


Batchtrai input_ids:
 tensor([[   28,  4065,  9502,  ...,     1,     1,  7592],
        [    1,   470,    88,  ...,     1,     1,     1],
        [    1,   121,    34,  ...,     1,     1,     1],
        ...,
        [   13,  1277,  1456,  ...,     1,     1,     1],
        [11055,     1,  6193,  ...,     1,     1,     1],
        [    1,  1215,    51,  ...,     1,     1,     1]])
Batch labels: tensor([1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0,
        1, 1, 1, 0, 0, 1, 1, 1])


In [27]:
from torch import nn
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size, num_layers=1):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)  # No activation

    def forward(self, x):
        x = self.embedding(x)
        _, hidden = self.rnn(x)
        out = self.fc(hidden[-1])  # No activation, raw logits
        return out


In [28]:
model=RNNModel(1000,100,128,1)

In [29]:
print("Model Parameters:")
for name, param in model.named_parameters():
    print(f"Name: {name}")
    print(f"  Shape: {param.shape}")
    print(f"  Requires Grad: {param.requires_grad}\n")

Model Parameters:
Name: embedding.weight
  Shape: torch.Size([1000, 100])
  Requires Grad: True

Name: rnn.weight_ih_l0
  Shape: torch.Size([128, 100])
  Requires Grad: True

Name: rnn.weight_hh_l0
  Shape: torch.Size([128, 128])
  Requires Grad: True

Name: rnn.bias_ih_l0
  Shape: torch.Size([128])
  Requires Grad: True

Name: rnn.bias_hh_l0
  Shape: torch.Size([128])
  Requires Grad: True

Name: fc.weight
  Shape: torch.Size([1, 128])
  Requires Grad: True

Name: fc.bias
  Shape: torch.Size([1])
  Requires Grad: True



In [30]:
len(vocab)

42784

In [31]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [32]:
def accuracy(preds, y):

    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [33]:
# Model parameters
vocab_size = len(vocab)
embed_size = 100
hidden_size = 128
output_size = 1
num_epochs = 12
learning_rate = 0.001

# Initialize model, loss, and optimizer
model = RNNModel(vocab_size, embed_size, hidden_size, output_size)
model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_acc = 0.0
    for batch in train_dataloader:

        optimizer.zero_grad()

        # Forward pass
        batch['input_ids'] = batch['input_ids'].to(device)
        outputs = model(batch['input_ids'])

        # Ensure labels have the same shape as outputs
        labels = batch['label'].unsqueeze(1).float()  # Reshape labels

        # Calculate loss
        loss = criterion(outputs, labels)
        total_loss += loss.item()

        # Calculate accuracy
        acc = accuracy(outputs, labels)
        total_acc += acc.item()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
    print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy: {acc.item()}")



Epoch [1/12], Loss: 0.6990914344787598
Epoch [1/12], Accuracy: 0.5


KeyboardInterrupt: 

In [36]:
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 42.7 MB/s eta 0:00:00


In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from pytorch_lightning import LightningModule, Trainer
from torch.utils.data import DataLoader, Dataset

# Define the RNN model as a LightningModule
class RNNLightningModel(LightningModule):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size, learning_rate):
        super().__init__()
        self.save_hyperparameters()  # Save hyperparameters for easy access
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, input_ids):
        embeds = self.embedding(input_ids)
        _, (hidden, _) = self.rnn(embeds)
        outputs = self.fc(hidden[-1])  # Use the last hidden state
        return outputs

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        labels = batch['label'].unsqueeze(1).float()
        outputs = self(input_ids)
        loss = self.criterion(outputs, labels)

        # Calculate accuracy
        preds = torch.sigmoid(outputs) > 0.5
        acc = (preds == labels).float().mean()

        # Log metrics
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("train_acc", acc, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        labels = batch['label'].unsqueeze(1).float()
        outputs = self(input_ids)
        loss = self.criterion(outputs, labels)

        # Calculate accuracy
        preds = torch.sigmoid(outputs) > 0.5
        acc = (preds == labels).float().mean()

        # Log metrics
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_acc", acc, on_step=False, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

# Custom Dataset
class CustomImdbDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        text = self.dataset[index]["text"]
        label = self.dataset[index]["label"]
        encoded_text = self.tokenizer.encode(text)
        return {
            "input_ids": torch.tensor(encoded_text, dtype=torch.long),
            "label": torch.tensor(label, dtype=torch.long),
        }

# Example tokenizer
class CustomTokenizer:
    def __init__(self, vocab):
        self.vocab = vocab
        self.unk_token = "<UNK>"

    def tokenize(self, text):
        return text.lower().split()

    def encode(self, text):
        tokens = self.tokenize(text)
        return [self.vocab.get(token, self.vocab[self.unk_token]) for token in tokens]

# Custom collate function for padding
def custom_collate_fn(batch):
    input_ids = [item["input_ids"] for item in batch]
    labels = [item["label"] for item in batch]
    max_len = max(len(seq) for seq in input_ids)

    padded_input_ids = torch.stack([
        torch.cat([seq, torch.full((max_len - len(seq),), 0, dtype=torch.long)])
        for seq in input_ids
    ])

    labels = torch.stack(labels)
    return {"input_ids": padded_input_ids, "label": labels}

# Initialize tokenizer, dataset, and dataloader

tokenizer = CustomTokenizer(vocab)

# Training and validation datasets


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:


train_data = CustomImdbDataset(train_preprocessed, tokenizer)
val_data = CustomImdbDataset(valid_preprocessed, tokenizer)

train_dataloader = DataLoader(train_data, batch_size=64, collate_fn=custom_collate_fn)
val_dataloader = DataLoader(val_data, batch_size=64, collate_fn=custom_collate_fn)

# Hyperparameters
vocab_size = len(vocab)
embed_size = 100
hidden_size = 128
output_size = 1
learning_rate = 0.001

# Initialize the model
model = RNNLightningModel(vocab_size, embed_size, hidden_size, output_size, learning_rate)

# Train the model with PyTorch Lightning's Trainer
trainer = Trainer(max_epochs=12, log_every_n_steps=1)
trainer.fit(model, train_dataloader, val_dataloader)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | embedding | Embedding         | 4.3 M  | train
1 | rnn       | LSTM              | 117 K  | train
2 | fc        | Linear            | 129    | train
3 | criterion | BCEWithLogitsLoss | 0      | train
--------------------------------------------------------
4.4 M     Trainable params
0         Non-trainable params
4.4 M     Total params
17.585    Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]